In [1]:
!ls /datasets/usc-drive/"DSO 510 - Food Deserts"/IHME_USA_COUNTY_CVD_MORTALITY_RATES_1980_2014_NATIONAL_Y2017M05D16.XLSX

'/datasets/usc-drive/DSO 510 - Food Deserts/IHME_USA_COUNTY_CVD_MORTALITY_RATES_1980_2014_NATIONAL_Y2017M05D16.XLSX'


In [103]:
import pandas as pd

def merge_health_data(path: str):
    """
    Takes input file path, merges all sheets of IHME file, and writes result back to same drive
    """
    global df
    sheet_to_df_map = pd.read_excel(path, sheet_name=None, engine='openpyxl')

    for idx, key in enumerate(list(sheet_to_df_map.keys())):
        print(f'On sheet {idx+1}/{len(list(sheet_to_df_map.keys()))}', end='\r')
        temp = pd.read_excel(path, sheet_name=key, engine='openpyxl', skiprows=1)

        # remove states & header rows
        temp = temp[temp['FIPS'] >= 1000]
        # separate state & county into 2 columns
        temp[['county', 'state']] = temp.loc[:,'Location'].str.split(', ', 1, expand=True)
        # clean & reshape data
        temp = temp.drop('Location', axis=1) \
            .melt(id_vars=["FIPS", "state", "county"], 
                    var_name="Date", 
                    value_name="Value") \
            .query("Date.str.contains('Change')==False")

        temp["mortality"] = temp.Value.apply(lambda x: float(x.split(' ')[0]))
        temp["lower_95"] = temp.Value.apply(lambda x: float(x.split(' ')[1].replace('(', '').replace(',', '')))
        temp["upper_95"] = temp.Value.apply(lambda x: float(x.split(' ')[2].replace(')', '')))
        temp['year'] = temp.Date.apply(lambda x: int(x.split(',')[1].replace('*', '')))
        temp = temp[['FIPS', 'state', 'county', 'year', "mortality", "lower_95", "upper_95"]]
        temp['disease'] = key
        
        # initialize result in 1st iteration, concat after
        if idx == 0:
            df = temp
        else:
            temp = temp[[col for col in temp.columns if 'county' not in col and 'state' not in col]]
            df = pd.concat([df, temp], axis=0)
            
    #write back to path      
    df.to_excel("/".join(s.split('/')[:-1]) + "/health_data_clean.xlsx", index=False)

    return f'Wrote result back to {"/".join(s.split("/")[:-1]) + "/health_data_clean.xlsx"}'
     


In [104]:
merge_health_data('/datasets/usc-drive/DSO 510 - Food Deserts/IHME_USA_COUNTY_CVD_MORTALITY_RATES_1980_2014_NATIONAL_Y2017M05D16.XLSX')

'Wrote result back to /datasets/usc-drive/DSO 510 - Food Deserts/health_data_clean.xlsx'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a805a7fe-fe3d-44c2-a936-f271a2a4c12f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>